In [40]:
import requests
import random
import re

In [36]:
#@title board utils
from os import kill
def generate_board(n = 25, lang = "eng", c = 5, k = 1):
  '''
  inputs:
    n (int): number of words
    lang (str): language of dictionary (must be on the github folder)
    c (int): number of colored words for each team
    k (int): number of killer words

  returns a dictionary such that dict[word] = color
  '''

  url = f"https://raw.githubusercontent.com/mich1803/Codenames-LLM/main/wordlists/{lang}.txt"

  try:
    # Fetch the content of the file
    response = requests.get(url)
  except:
    raise Exception("ERROR: The language entered is wrong or has no dictionary in the github folder")
  words = response.text.splitlines()

  # Select n unique random words
  random_words = random.sample(words, n)

  # Select indices for the colors
  indices = list(range(n))
  blue_indices = random.sample(indices, c)
  remaining_indices = [i for i in indices if i not in blue_indices]
  red_indices = random.sample(remaining_indices, c)
  remaining_indices = [i for i in remaining_indices if i not in red_indices]
  black_index = random.sample(remaining_indices, k)

  if lang == "eng":
    colors = [
      "blue" if i in blue_indices else
      "red" if i in red_indices else
      "killer" if i in black_index else
      "neutral"
      for i in range(25)
      ]
  elif lang == "ita":
    colors = [
      "blu" if i in blue_indices else
      "rosso" if i in red_indices else
      "assassino" if i in black_index else
      "neutrale"
      for i in range(25)
      ]

  # Create a dictionary with list comprehension
  word_color_dict = {random_words[i]: colors[i] for i in range(25)}
  return word_color_dict

def board4prompt(word_color_dict, lang = "eng", master = False):
  '''
  inputs:
    word_color_dict(dict): board dict generated from generate_board()
    lang (str): language of prompt

  return a string version for the prompt for LLMs
  '''

  if lang == "eng":
    init = "The actual word board is: \n"
  elif lang == "ita":
    init = "La board al momento è: \n"
  else: raise Exception("ERROR: The language entered is wrong or has no dictionary in the github folder")

  if master:
    return init + (" | ". join([f"{i} ({word_color_dict[i]})" for i in word_color_dict])) + "\n"
  else:
    return init + (" | ".join(word_color_dict)) + "\n"

In [39]:
board = generate_board(lang = "ita")
board_prompt = board4prompt(board, lang="ita", master = True)
print(board_prompt)

La board al momento è: 
PERSONAGGIO (neutrale) | MOGLIE (blu) | FERRO (rosso) | MADRE (neutrale) | RISPOSTA (rosso) | FAMIGLIA (rosso) | OMBRA (neutrale) | PALAZZO (neutrale) | MASSA (neutrale) | MOVIMENTO (neutrale) | CRISI (neutrale) | LUNA (blu) | FOGLIA (neutrale) | PIACERE (neutrale) | ISOLA (neutrale) | CAFFÈ (neutrale) | PELLE (neutrale) | PADRE (rosso) | VOLTO (blu) | PASSO (blu) | FORZA (blu) | CUORE (assassino) | ORECCHIO (neutrale) | AMORE (rosso) | COLPO (neutrale)



cose da implementare ancora per la gestione del gioco:
*   funzione per leggere il prompt di indizio del master LLM (magari chiedere all'LLM di scrivere la risposta in un modo particolare nel prompt, esempio: dai l'indizio di una parola e un numero nel formato {[(parola)(numero)]})
*   funzione per leggere il prompt di risposta dell'indovino LLM
*   funzione per la partita: finisce se una squadra prende tutte le sue carte o se la carta killer viene scoperta, la partita restituisce: una tupla (vincitore, metodo di vittoria, round)
*   fare le statistiche su quale metodo di vittoria è più frequente, in quanti round finisce mediamente la partita, confrontarli tra LLM, lingue, numero di carte ecc.



In [38]:
#@title prompts

START_master_1 = {"ita": "Stiamo giocando al gioco da tavola NOMI IN CODICE, il tuo ruolo sarà quello del master. \n",
                  "eng": "We are playing CODENAMES, you are going to be the master."}

START_master_2 = {"ita": "I tuoi compagni di squadra non vedranno il colore delle parole, tu devi fornirgli: \n Un indizio di una parola per fargli indovinare le parole del colore della tua squadra, \n Un numero intero che rappresenta quante parole sono in relazione con l'indizio che hai appena dato. \n ATTENZIONE: se la tua squadra nomina la parola assassina perde all'istante. \n",
                  "eng": "Your teammates will not see the color of the words, you must provide them with: \n A one-word clue to help them guess the words of your team's color, \n An integer that represents how many words are related to the clue you just gave. BEWARE: if your team says the killer word you lose instantly\n"}

team4prompt = {('eng','r'): "You are on the RED team. \n",
               ('eng','b'): "You are on the BLUE team. \n",
               ('ita', 'r'): "Fai parte della squadra ROSSA. \n",
               ('ita', 'b'): "Fai parte della squadra BLU. \n"}

FORMAT_master = {"ita": "Scrivi la tua risposta nel formato: {(inidizio di una parola),(numero)}",
                  "eng": "Write your answer in the format: {(one-word clue),(number)}"}

#IL PROMPT INIZIALE DEL MASTER SARA': START_master_1 + master_board + START_master_2 + team4prompt + FORMAT_master

START_guesser = {"ita": "Stiamo giocando al gioco da tavola NOMI IN CODICE, il tuo ruolo sarà quello del guesser. Date le parole sulla board e all'indizio del tuo master devi cercare di indovinare le parole della tua squadra. Se indovini e il tuo master ha dato un numero maggiore di uno puoi fare un altro tentativo.\n",
                 "eng": "We are playing the board game CODENAMES, and your role will be that of the guesser. Given the words on the board and the clue from your spymaster, you must try to guess your team's words. If you guess correctly and your spymaster has given a number greater than one, you can make another attempt.\n"}

FORMAT_guesser = {"ita": "Scrivi la tua risposta nel formato: {risposta }",
                  "eng": "Write your answer in the format: {one-word guess}"}

MtoG_clue = {
    "ita": "Il tuo master ha dato l'indizio: ",
    "eng": "The master gave the clue: "
}

MtoG_number = {
    "ita": " è il numero. \n",
    "eng": " is the number. \n"
}



In [44]:
#@title text-handling utils
def read_clue(answer):
  # Regular expression to match the format "{(one-word clue),(number)}"
    match = re.search(r"\{\((\w+)\),\((\d+)\)\}", answer)
    if match:
        clue = match.group(1)
        number = int(match.group(2))
        return (clue, number)
    else:
        raise ValueError("The response format is incorrect")

def read_guess(answer):
    # Regular expression to match the format "{(one_word guess)}"
    match = re.search(r"\{\((\w+)\)\}", answer)
    if match:
        clue = match.group(1)
        return clue
    else:
        raise ValueError("The response format is incorrect")

In [ ]:
def play_game_GPT(lang, n, c, k, verbose = False):

  r = 0
  turn =  random.choice(["r", "b"])
  game = True
  board = generate_board(n, lang, c, k)
  red_points = 0
  blue_point = 0

  while game:
    if verbose: print(f" --- ROUND {r} ---")

    if r == 0: #fare un controllo sul numero di round per squadra

        #MASTER FIRST TEAM
      #API -> apri conversazione con GPT master 1
      prompt_master_1 = START_master_1[lang] + board4prompt(board, lang, master = True) + START_master_2[lang] + team4prompt[(lang,turn)] + FORMAT_master[lang]
      if verbose: print(f"NARRATOR to MASTER({turn}): {prompt_master_1}")
      master_answer_1 = "" ### da GPT
      if verbose: print(f"MASTER({turn}): {master_answer_1}")
      try:
        clue_1, n_1 = read_clue(master_answer_1)
      except:
        if verbose: print("!!! ERROR (Incorrect format of answer)")
        w = "b" if turn == "r" else "r"
        return (w, "format", r)

        #GUESSER FIRST TEAM
      #API -> apri conversazione con GPT guesser 1
      prompt_guesser_1 = START_guesser[lang] + board4prompt(board, lang) + team4prompt[((lang,turn))] + MtoG_clue[lang] + clue_1 + ", " + MtoG_number[lang] + FORMAT_guesser[lang]
      if verbose: print(f"NARRATOR to GUESSER({turn}): {prompt_guesser_1}")
      guesser_answer_1 = "" ### da GPT
      if verbose: print(f"GUESSER({turn}): {guesser_answer_1}")
      try:
        guess = read_guess(guesser_answer_1).upper()
      except:
        if verbose: print("!!! ERROR (Incorrect format of answer)")
        w = "b" if turn == "r" else "r"
        return (w, "format", r)

        #EVALUATE GUESS
      try:
        x = board[guess]
      except: # la guess non è nella board
        if verbose: print("!!! ERROR (Guess non in board)")
        w = "b" if turn == "r" else "r"
        return (w, "format", r)
      if x == "killer": # la guess è la killer
        if verbose: print(f"NARRATOR: The killer word have been selected, the game ends.")
        w = "b" if turn == "r" else "r"
        return (w, "killer", r)
      elif x == "blue": # la guess è blue
        blue_points += 1
        if verbose: print(f"NARRATOR: A blue word have been selected (blue score = {blue_points}).")
        del board[guess]
        if blue_points == c:
          if verbose: print(f"NARRATOR: The blue team reached the goal, the game ends.")
          return ("b", "win", r)
      elif x == "red": # la guess è red
        red_points += 1
        if verbose: print(f"NARRATOR: A red word have been selected (red score = {red_points}).")
        del board[guess]
        if red_points == c:
          if verbose: print(f"NARRATOR: The red team reached the goal, the game ends.")
          return ("r", "win", r)
      else: #la guess è neutrale
        if verbose: print(f"NARRATOR: A neutral word have been selected.")

        ## MASTER AND GUESSER of the other team
      #vedere come gestire gli altri

